In [238]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import camelot
import tabula
import warnings
import codecs
import asyncio
import threading

warnings.simplefilter('ignore', UserWarning)
pd.set_option('display.max_rows', 100)
codecs.register_error('none', lambda e: ('', e.end))

In [323]:
#camelot or tabulaの試し①

start_url = 'http://ke.kabupro.jp/code/9946.htm'
response = requests.get(start_url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table', attrs={'class': 'Quote'})
if 'リクエストされたページがみつかりませんでした' in soup.text:
    pass
else:
    try:
        for i in range(0, 5):
            print(i)
            if '訂正' in table.find_all('td', text = re.compile('.*決算短信.*'))[i].text\
            or '補足' in table.find_all('td', text = re.compile('.*決算短信.*'))[i].text:
                pass
            else:
                target_pdf = table.find_all('td', text = re.compile('.*決算短信.*'))[i].find('a').get('href')
                break
    #最近上場した企業は、決算短信がまだ存在しない
    except Exception as e:
            print('決算短信が存在しない')
            print(e)
            print('\n')
            pass
        
target_pdf

0
1


'http://ke.kabupro.jp/tsp/20210408/140120210408491308.pdf'

In [324]:
'''
https://camelot-py.readthedocs.io/en/master/api.html
table_areas (list, optional (default: None))
– List of table area strings of the form x1,y1,x2,y2 where (x1, y1) -> left-top 
and (x2, y2) -> right-bottom in PDF coordinate space.
'''
#camelot or tabulaの試し①

#1663['0,700,600,40'], 
try:
    #tables = camelot.read_pdf(target_pdf, flavor='stream', pages = '4-end', table_areas=['0,800,700,0'])
    df_tbls = tabula.read_pdf(target_pdf, pages = 'all', stream=True, silent=True, )
except Exception as e:
    print('--------------------')
    print('取込エラー')
    print(e)
    print('--------------------')
    pass
#PDFの座標の確認
#camelot.plot(tables[0], kind='textedge').show()

In [331]:
df_tbls[9]

,Unnamed: 0,前連結会計年度,当連結会計年度
0,NaN,(2020年2月29日),(2021年2月28日)
1,負債の部,NaN,NaN
2,流動負債,NaN,NaN
3,買掛金,"11,107","10,947"
4,加盟店買掛金,"20,341","18,832"
5,加盟店借勘定,616,550
6,短期借入金,"2,074","4,198"
7,1年内返済予定の長期借入金,-,"1,422"
8,リース債務,"4,851","4,314"
9,未払金,"6,628","6,092"


In [326]:
#camelotまたはtabulaの試し①

#流動資産合計	現金及び預金	現金預金	有形固定資産	有形固定資産合計	投資有価証券	投資その他の資産合計	流動負債合計	固定負債合計
feature_list = ['流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', '投資有価証券', 
            '投資その他の資産合計', '流動負債合計', '固定負債合計']
feature_remove_list = []
df_elements = []
#for table in tables:
#    df_tbl = table.df
    
for df_tbl in df_tbls:    
    for feature in feature_list:
        if len(df_tbl[df_tbl.iloc[:,0] == feature]) == 1:
            #資産と負債が違うテーブルのため、df_element1の列がずれる場合がある
            #特徴量と値だけの1行2列のデータフレームを作成
            df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-1]]
            print(df_element1)
            #後のconcatで列がずれないようにするため、値の列の名前を1に統一する
            df_element2 = df_element1.rename(columns={df_element1.columns[1]: 1})
            df_elements.append(df_element2)
            #資産と負債のテーブルが2つある銘柄がある（連結と単独のテーブル）
            #同じ特徴量を2回取得することを避けるため、一度入れた特徴量は削除リストに登録
            feature_remove_list.append(feature)
    #テーブル処理が1つ終わったら、削除リストに登録された特徴量を元のリストから削除する
    if len(feature_remove_list) != 0:
        for feature_remove in feature_remove_list:
            feature_list.remove(feature_remove)
        feature_remove_list = []
        
df_all1 = pd.concat(df_elements).T
df_all2 = pd.DataFrame([list(df_all1.iloc[1,:])], columns=df_all1.iloc[0,:],)
#df_all3 = df_all2.copy()
#df_all3['code'] = 1301



   Unnamed: 0 当連結会計年度
17     流動負債合計  63,433
   Unnamed: 0 当連結会計年度
26     固定負債合計  12,000
  Unnamed: 0    当連結会計年度
2     現金及び預金  15,278百万円


In [ ]:
dict = {'code':[], 'error':'test'}
dict['new'] = ['ee']
dict['new'].append('ss')
dict['new']

try:
    dict['new2'].append('ee')
except:
    dict['new2'] = ['ee']
    
try:
    dict['new2'].append('ff')
except:
    dict['new2'] = ['ff']    
    
dict

In [49]:
#camelotで取得した表の加工　余分な文字を消す
df_camelot2 = pd.read_csv('temp_file/全銘柄PDFデータ（camelot加工前）.csv', encoding=("cp932"), header=0)

replace_feature_list = ['流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', '投資有価証券', 
                '投資その他の資産合計', '流動負債合計', '固定負債合計']
replace_list = [',', '△', '―', '-', '，', '\n', ' ',
                '※1', '※2', '※3', '※4', '※5', '※6', '※7', '※8', '※9',
                '※１', '※２', '※３', '※４', '※５', '※６', '※７', '※８', '※９']
for i in replace_feature_list:
    for j in replace_list:
        df_camelot2[i] = df_camelot2[i].str.replace(j, '')
df_camelot2

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1301,86649,7100,NaN,NaN,15715,11682,13547,39189,37166
1,1332,231751,10986,NaN,NaN,147428,38076,86443,153467,134021
2,1352,12137,4019,NaN,NaN,26593,345,2390,11866,21666
3,1375,10202,NaN,NaN,18031,NaN,NaN,NaN,7614,NaN
4,1376,28828145,3447948,NaN,NaN,6627416,NaN,2829597,16514107,1726482
...,...,...,...,...,...,...,...,...,...,...
2914,9990,17545707,1132167,NaN,NaN,6393571,4627857,13423191,5951278,5038906
2915,9991,71446,20.8,NaN,1.4,23439,0.0,7290,37959,5094
2916,9994,32537,8664,NaN,NaN,11459,894,11165,28133,5102
2917,9995,28864,3395,NaN,NaN,1537,2131,3112,10263,888


In [50]:
#十分に取り込めていない銘柄コードについて、
#①個別に抽出する銘柄コードの抽出と、②削除する銘柄コードの抽出
camelot_2nd_dict = {}
drop_list = []
for i in range(0, len(df_camelot2)):
    #①個別に抽出する銘柄コードの抽出
    if len(df_camelot2.iloc[i, :].value_counts()) == 1:
        camelot_2nd_dict[df_camelot2.iloc[i, 0]] = '資料が左に細くて1つも取り込めていない？'
        print('code=', df_camelot2.iloc[i, 0], '個別抽出：資料が左に細くて1つも取り込めない')
    elif df_camelot2.iloc[i, 9] == '非流動負債合計':
        camelot_2nd_dict[df_camelot2.iloc[i, 0]] = '固定負債合計の値が非流動負債合計'
        print('code=', df_camelot2.iloc[i, 0], '個別抽出：固定負債合計の値が非流動負債合計')
        
for i in range(0, len(df_camelot2)):    
    #②削除する銘柄コードの抽出
    if len(df_camelot2.iloc[i, :].value_counts()) <= 3:
        drop_list.append(df_camelot2.iloc[i, 0])
        print('code=', df_camelot2.iloc[i, 0], '削除：特徴量が2つ以下')
    elif df_camelot2.iloc[i, :].str.contains('%').sum() !=0:
        drop_list.append(df_camelot2.iloc[i, 0])
        print('code=', df_camelot2.iloc[i, 0], '削除：%を含む')
    elif df_camelot2.iloc[i, 9] == '非流動負債合計':
        drop_list.append(df_camelot2.iloc[i, 0])
        print('code=', df_camelot2.iloc[i, 0], '削除：固定負債合計の値が非流動負債合計')
        
df_camelot2

code= 3277 個別抽出：資料が左に細くて1つも取り込めない
code= 3402 個別抽出：固定負債合計の値が非流動負債合計
code= 4183 個別抽出：固定負債合計の値が非流動負債合計
code= 6232 個別抽出：資料が左に細くて1つも取り込めない
code= 6361 個別抽出：固定負債合計の値が非流動負債合計
code= 7203 個別抽出：固定負債合計の値が非流動負債合計
code= 7605 個別抽出：資料が左に細くて1つも取り込めない
code= 8160 個別抽出：資料が左に細くて1つも取り込めない
code= 8399 個別抽出：資料が左に細くて1つも取り込めない
code= 8616 個別抽出：資料が左に細くて1つも取り込めない
code= 8746 個別抽出：資料が左に細くて1つも取り込めない
code= 9201 個別抽出：固定負債合計の値が非流動負債合計
code= 9519 個別抽出：固定負債合計の値が非流動負債合計
code= 9678 個別抽出：資料が左に細くて1つも取り込めない
code= 9719 個別抽出：固定負債合計の値が非流動負債合計
code= 3245 削除：%を含む
code= 3277 削除：特徴量が2つ以下
code= 3402 削除：固定負債合計の値が非流動負債合計
code= 4095 削除：特徴量が2つ以下
code= 4183 削除：固定負債合計の値が非流動負債合計
code= 4519 削除：特徴量が2つ以下
code= 4689 削除：特徴量が2つ以下
code= 4755 削除：特徴量が2つ以下
code= 5202 削除：特徴量が2つ以下
code= 5464 削除：特徴量が2つ以下
code= 6098 削除：特徴量が2つ以下
code= 6232 削除：特徴量が2つ以下
code= 6361 削除：固定負債合計の値が非流動負債合計
code= 6417 削除：特徴量が2つ以下
code= 6594 削除：特徴量が2つ以下
code= 6697 削除：特徴量が2つ以下
code= 6758 削除：特徴量が2つ以下
code= 6981 削除：特徴量が2つ以下
code= 7150 削除：特徴量が2つ以下
code= 7157 削除：特徴量が2つ以下
code= 7167 削除：特徴量

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1301,86649,7100,NaN,NaN,15715,11682,13547,39189,37166
1,1332,231751,10986,NaN,NaN,147428,38076,86443,153467,134021
2,1352,12137,4019,NaN,NaN,26593,345,2390,11866,21666
3,1375,10202,NaN,NaN,18031,NaN,NaN,NaN,7614,NaN
4,1376,28828145,3447948,NaN,NaN,6627416,NaN,2829597,16514107,1726482
...,...,...,...,...,...,...,...,...,...,...
2914,9990,17545707,1132167,NaN,NaN,6393571,4627857,13423191,5951278,5038906
2915,9991,71446,20.8,NaN,1.4,23439,0.0,7290,37959,5094
2916,9994,32537,8664,NaN,NaN,11459,894,11165,28133,5102
2917,9995,28864,3395,NaN,NaN,1537,2131,3112,10263,888


In [51]:
#削除する銘柄コードの削除
for i in drop_list:
    df_camelot2.drop(df_camelot2.index[df_camelot2['code'] == i], inplace=True)
df_camelot2

,code,流動資産合計,現金及び預金,現金預金,有形固定資産,有形固定資産合計,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,1301,86649,7100,NaN,NaN,15715,11682,13547,39189,37166
1,1332,231751,10986,NaN,NaN,147428,38076,86443,153467,134021
2,1352,12137,4019,NaN,NaN,26593,345,2390,11866,21666
3,1375,10202,NaN,NaN,18031,NaN,NaN,NaN,7614,NaN
4,1376,28828145,3447948,NaN,NaN,6627416,NaN,2829597,16514107,1726482
...,...,...,...,...,...,...,...,...,...,...
2914,9990,17545707,1132167,NaN,NaN,6393571,4627857,13423191,5951278,5038906
2915,9991,71446,20.8,NaN,1.4,23439,0.0,7290,37959,5094
2916,9994,32537,8664,NaN,NaN,11459,894,11165,28133,5102
2917,9995,28864,3395,NaN,NaN,1537,2131,3112,10263,888


In [44]:
df_camelot2.to_csv('temp_file/全銘柄PDFデータ（camelot加工済み）.csv', mode='w', index=False, encoding="shift-jis", errors='none')


In [80]:
#個別対応①　camelot「固定負債合計の値が非流動負債合計」PDFデータの取込み
feature_list = ['流動資産合計', '現金及び預金', '現金預金', '有形固定資産', '有形固定資産合計', '投資有価証券', 
            '投資その他の資産合計', '流動負債合計', '固定負債合計']
feature_remove_list = []
df_elements = []
for table in tables:
    df_tbl = table.df
    
#for df_tbl in df_tbls:    
    for feature in feature_list:
        if len(df_tbl[df_tbl.iloc[:,0] == feature]) == 1:
            #資産と負債が違うテーブルのため、df_element1の列がずれる場合がある
            #特徴量と値だけの1行2列のデータフレームを作成
            df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-1]]
            #左から2列目に値が入っているPDFもある
            if df_element1.iloc[0, 1] == '':
                df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-2]]
            #後のconcatで列がずれないようにするため、値の列の名前を1に統一する
            df_element2 = df_element1.rename(columns={df_element1.columns[1]: 1})
            df_elements.append(df_element2)
            #資産と負債のテーブルが2つある銘柄がある（連結と単独のテーブル）
            #同じ特徴量を2回取得することを避けるため、一度入れた特徴量は削除リストに登録
            feature_remove_list.append(feature)
    #テーブル処理が1つ終わったら、削除リストに登録された特徴量を元のリストから削除する
    if len(feature_remove_list) != 0:
        for feature_remove in feature_remove_list:
            feature_list.remove(feature_remove)
        feature_remove_list = []
        
df_all1 = pd.concat(df_elements).T
df_all2 = pd.DataFrame([list(df_all1.iloc[1,:])], columns=df_all1.iloc[0,:],) 


,流動資産合計,現金及び預金,投資有価証券,投資その他の資産合計,流動負債合計,固定負債合計
0,"3,257,717","1,891,731","646,888","675,887","432,941","3,346"


In [103]:
#個別対応②　camelot「資料が左に細くて1つも取り込めない」PDFデータの取込み

#現金及び預金→現金及び現金同等物、固定負債合計→非流動負債合計に変更
feature_list = ['流動資産合計', '現金及び現金同等物', '現金預金', '有形固定資産', '有形固定資産合計', '投資有価証券', 
            '投資その他の資産合計', '流動負債合計', '非流動負債合計']
feature_remove_list = []
df_elements = []
for table in tables:
    df_tbl = table.df
    
#for df_tbl in df_tbls:    
    for feature in feature_list:
        if len(df_tbl[df_tbl.iloc[:,0] == feature]) == 1:
            #資産と負債が違うテーブルのため、df_element1の列がずれる場合がある
            #特徴量と値だけの1行2列のデータフレームを作成
            df_element1 = df_tbl[df_tbl.iloc[:,0] == feature].iloc[:, [0,-1]]
            #後のconcatで列がずれないようにするため、値の列の名前を1に統一する
            df_element2 = df_element1.rename(columns={df_element1.columns[1]: 1})
            df_elements.append(df_element2)
            #資産と負債のテーブルが2つある銘柄がある（連結と単独のテーブル）
            #同じ特徴量を2回取得することを避けるため、一度入れた特徴量は削除リストに登録
            feature_remove_list.append(feature)
    #テーブル処理が1つ終わったら、削除リストに登録された特徴量を元のリストから削除する
    if len(feature_remove_list) != 0:
        for feature_remove in feature_remove_list:
            feature_list.remove(feature_remove)
        feature_remove_list = []
        
df_all1 = pd.concat(df_elements).T
df_all2 = pd.DataFrame([list(df_all1.iloc[1,:])], columns=df_all1.iloc[0,:],)\
.rename(columns={'現金及び現金同等物': '現金及び預金'}).rename(columns={'非流動負債合計': '固定負債合計'})

df_all2

,流動資産合計,現金及び預金,有形固定資産,流動負債合計,固定負債合計,投資有価証券
0,"468,136","136,084","160,722","291,212","82,087",
